# Loss and Metrics

## To Do
* check sinri and sdri

## Resources
* https://github.com/aliutkus/speechmetrics
* https://source-separation.github.io/tutorial/landing.html
* https://github.com/r06944010/Speech-Separation-TF2

In [1]:
from mir_eval.separation import bss_eval_sources
import tensorflow as tf
import numpy as np
import math
from itertools import permutations

import evaluation
import numpy as np
import tensorflow as tf
import itertools

In [2]:
# Batch size of 1
BATCH_SIZE = 1

s1 = np.array([1,2,1.5],dtype='float32')
s2 = np.array([3,4,4.5],dtype='float32')
s = np.stack([s1,s2])
s = np.reshape(s,(1,2,3))
mix_s = np.sum(np.stack([s1,s2]),axis=0)
mix_s = mix_s.reshape(1,3)

s1_est = np.array([1.4,1.8,1.7],dtype='float32')
s2_est = np.array([3.2,4.5,4.1],dtype='float32')
s_est = np.stack([s1_est,s2_est])
s_est = np.reshape(s_est,(1,2,3))

print(s1.shape)
print(s2.shape)
print(s.shape)
print(s_est.shape)
print(mix_s.shape)

(3,)
(3,)
(1, 2, 3)
(1, 2, 3)
(1, 3)


In [3]:
# Batch size of 2 copy 
BATCH_SIZE = 2

s1a = np.array([1,2,1.5],dtype='float32')
s2a = np.array([3,4,4.5],dtype='float32')
sa = np.stack([s1a,s2a])
sa = np.reshape(sa,(1,2,3))
mix_sa = np.sum(np.stack([s1a,s2a]),axis=0)

s1b = np.array([1,2,1.5],dtype='float32')
s2b = np.array([3,4,4.5],dtype='float32')
sb = np.stack([s1b,s2b])
sb = np.reshape(sb,(1,2,3))
mix_sb = np.sum(np.stack([s1b,s2b]),axis=0)

mix_s = np.stack([mix_sa,mix_sb])
s = np.concatenate([sa,sb],axis=0)
print(mix_s.shape)
print(s.shape)

s1a_est = np.array([1.4,1.8,1.7],dtype='float32')
s2a_est = np.array([3.2,4.5,4.1],dtype='float32')

s1b_est = np.array([1.4,1.8,1.7],dtype='float32')
s2b_est = np.array([3.2,4.5,4.1],dtype='float32')

sa_est = np.stack([s1a_est,s2a_est])
sa_est = np.reshape(sa_est,(1,2,3))

sb_est = np.stack([s1b_est,s2b_est])
sb_est = np.reshape(sb_est,(1,2,3))

s_est = np.concatenate([sa_est,sb_est],axis=0)
print(s_est.shape)

(2, 3)
(2, 2, 3)
(2, 2, 3)


In [4]:
# Batch size of 2
BATCH_SIZE = 2

s1a = np.array([1,2,1.5],dtype='float32')
s2a = np.array([3,4,4.5],dtype='float32')
sa = np.stack([s1a,s2a])
sa = np.reshape(sa,(1,2,3))
mix_sa = np.sum(np.stack([s1a,s2a]),axis=0)

s1b = np.array([3,2.5,5],dtype='float32')
s2b = np.array([1,2,4],dtype='float32')
sb = np.stack([s1b,s2b])
sb = np.reshape(sb,(1,2,3))
mix_sb = np.sum(np.stack([s1b,s2b]),axis=0)

mix_s = np.stack([mix_sa,mix_sb])
s = np.concatenate([sa,sb],axis=0)
print(mix_s.shape)
print(s.shape)


s1a_est = np.array([1.4,1.8,1.7],dtype='float32')
s2a_est = np.array([3.2,4.5,4.1],dtype='float32')

s1b_est = np.array([2.8,2,4.7],dtype='float32')
s2b_est = np.array([1.4,2.3,4.2],dtype='float32')

sa_est = np.stack([s1a_est,s2a_est])
sa_est = np.reshape(sa_est,(1,2,3))

sb_est = np.stack([s1b_est,s2b_est])
sb_est = np.reshape(sb_est,(1,2,3))

s_est = np.concatenate([sa_est,sb_est],axis=0)
print(s_est.shape)


(2, 3)
(2, 2, 3)
(2, 2, 3)


In [5]:
"""
def log10(x):
    numerator = tf.math.log(x)
    denominator = tf.math.log(tf.constant(10, dtype=numerator.dtype))
    return numerator / denominator

def sisnr(s, s_hat, do_expand=False, eps=1e-8):
    if do_expand:
        s = np.expand_dims(s, axis=0)
        s_hat = np.expand_dims(s_hat, axis=0)
    dot_product = tf.math.reduce_sum(s*s_hat, axis=1, keepdims=True)
    squares = tf.math.reduce_sum(s*s, axis=1, keepdims=True)
    s_target = s * dot_product / squares
    e_noise = s_hat - s_target
    s_target_squared = tf.math.reduce_sum(s_target*s_target, axis=1)
    e_noise_squared = tf.math.reduce_sum(e_noise*e_noise, axis=1)
    return 10*log10(s_target_squared / (e_noise_squared + eps))

def pit_sisnr(y_true, y_pred,n_sources=2):
    y_mix = tf.math.reduce_sum(y_true,axis=1,keepdims=False)

    sn, sn_hat = [], []
    for i in range(n_sources):
        sn.append(y_true[:,i,:])
        sn_hat.append(y_pred[:,i,:])
    
    if n_sources == 2:
        sisnr_perm0_spk0 = sisnr(sn[0], sn_hat[0])
        sisnr_perm0_spk1 = sisnr(sn[1], sn_hat[1])
        sisnr_perm0 = (sisnr_perm0_spk0 + sisnr_perm0_spk1) / 2

        sisnr_perm1_spk0 = sisnr(sn[0], sn_hat[1])
        sisnr_perm1_spk1 = sisnr(sn[1], sn_hat[0])
        sisnr_perm1 = (sisnr_perm1_spk0 + sisnr_perm1_spk1) / 2

        sisnr_perm_invariant = tf.math.maximum(sisnr_perm0, sisnr_perm1)
        
        print(sn[0].shape)
        sisnr_b1 = sisnr(sn[0],y_mix)
        sisnr_b2 = sisnr(sn[1],y_mix)
        avg_SISNRi = ((sisnr_perm0_spk0 - sisnr_b1) + (sisnr_perm0_spk1 - sisnr_b2)) / 2
        
        return tf.math.reduce_mean(sisnr_perm_invariant), -1*avg_SISNRi 
    
    elif n_sources == 3:
        sisnr_perm0_spk0 = sisnr(sn[0], sn_hat[0])
        sisnr_perm0_spk1 = sisnr(sn[1], sn_hat[1])
        sisnr_perm0_spk2 = sisnr(sn[2], sn_hat[2])
        sisnr_perm0 = (sisnr_perm0_spk0 + sisnr_perm0_spk1 + sisnr_perm0_spk2) / 3

        sisnr_perm1_spk0 = sisnr(sn[0], sn_hat[0])
        sisnr_perm1_spk1 = sisnr(sn[1], sn_hat[2])
        sisnr_perm1_spk2 = sisnr(sn[2], sn_hat[1])
        sisnr_perm1 = (sisnr_perm1_spk0 + sisnr_perm1_spk1 + sisnr_perm1_spk2) / 3
        
        sisnr_perm2_spk0 = sisnr(sn[0], sn_hat[1])
        sisnr_perm2_spk1 = sisnr(sn[1], sn_hat[0])
        sisnr_perm2_spk2 = sisnr(sn[2], sn_hat[2])
        sisnr_perm2 = (sisnr_perm2_spk0 + sisnr_perm2_spk1 + sisnr_perm2_spk2) / 3
        
        sisnr_perm3_spk0 = sisnr(sn[0], sn_hat[1])
        sisnr_perm3_spk1 = sisnr(sn[1], sn_hat[2])
        sisnr_perm3_spk2 = sisnr(sn[2], sn_hat[0])
        sisnr_perm3 = (sisnr_perm3_spk0 + sisnr_perm3_spk1 + sisnr_perm3_spk2) / 3
        
        sisnr_perm4_spk0 = sisnr(sn[0], sn_hat[2])
        sisnr_perm4_spk1 = sisnr(sn[1], sn_hat[0])
        sisnr_perm4_spk2 = sisnr(sn[2], sn_hat[1])
        sisnr_perm4 = (sisnr_perm4_spk0 + sisnr_perm4_spk1 + sisnr_perm4_spk2) / 3
        
        sisnr_perm5_spk0 = sisnr(sn[0], sn_hat[2])
        sisnr_perm5_spk1 = sisnr(sn[1], sn_hat[1])
        sisnr_perm5_spk2 = sisnr(sn[2], sn_hat[0])
        sisnr_perm5 = (sisnr_perm5_spk0 + sisnr_perm5_spk1 + sisnr_perm5_spk2) / 3
        
        sisnr_perm_invariant = tf.stack([sisnr_perm1,sisnr_perm2,sisnr_perm3,sisnr_perm4,sisnr_perm5])
        sisnr_perm_invariant = tf.math.reduce_max(sisnr_perm_invariant)
        return tf.math.reduce_mean(sisnr_perm_invariant)
"""

"""
IN THE WORKS, NOT STABLE!!!

@tf.function
def get_permutation_invariant_sisnr(spk0_estimate, spk1_estimate, spk0_groundtruth, spk1_groundtruth):
    perm0_spk0 = sisnr(spk0_groundtruth, spk0_estimate, do_expand=True)
    perm0_spk1 = sisnr(spk1_groundtruth, spk1_estimate, do_expand=True)
    perm1_spk0 = sisnr(spk0_groundtruth, spk1_estimate, do_expand=True)
    perm1_spk1 = sisnr(spk1_groundtruth, spk0_estimate, do_expand=True)

    # Get best permutation
    if perm0_spk0 + perm0_spk1 > perm1_spk0 + perm1_spk1:
        return perm0_spk0, perm0_spk1

    return perm1_spk0, perm1_spk1 


@tf.function
def permutation_invariant_loss(y_true, y_pred,n_sources=2):
    # PIT for n-sources, work but very slow, needs to use tf.while_loop
    # yet, implementing tf.while_loop is a nightmare
    sn, sn_hat = [], []
    for i in range(n_sources):
        sn.append(y_true[:,i,:])
        sn_hat.append(y_pred[:,i,:])
        
    perm = list(permutations(range(n_sources)))
    sisnr_perm = []
        
    for i in range(len(perm)):
        sisnr_perm_spk = [sisnr(sn[p], sn_hat[j]) for p,j in enumerate(perm[i])]
        sisnr_perm.append(tf.math.reduce_sum(sisnr_perm_spk)/n_sources)

    sisnr_perm = tf.stack(sisnr_perm) 
    return -tf.math.reduce_max(sisnr_perm)
"""

'\nIN THE WORKS, NOT STABLE!!!\n\n@tf.function\ndef get_permutation_invariant_sisnr(spk0_estimate, spk1_estimate, spk0_groundtruth, spk1_groundtruth):\n    perm0_spk0 = sisnr(spk0_groundtruth, spk0_estimate, do_expand=True)\n    perm0_spk1 = sisnr(spk1_groundtruth, spk1_estimate, do_expand=True)\n    perm1_spk0 = sisnr(spk0_groundtruth, spk1_estimate, do_expand=True)\n    perm1_spk1 = sisnr(spk1_groundtruth, spk0_estimate, do_expand=True)\n\n    # Get best permutation\n    if perm0_spk0 + perm0_spk1 > perm1_spk0 + perm1_spk1:\n        return perm0_spk0, perm0_spk1\n\n    return perm1_spk0, perm1_spk1 \n\n\n@tf.function\ndef permutation_invariant_loss(y_true, y_pred,n_sources=2):\n    # PIT for n-sources, work but very slow, needs to use tf.while_loop\n    # yet, implementing tf.while_loop is a nightmare\n    sn, sn_hat = [], []\n    for i in range(n_sources):\n        sn.append(y_true[:,i,:])\n        sn_hat.append(y_pred[:,i,:])\n        \n    perm = list(permutations(range(n_sources)

In [6]:
def pit_loss(y_true, y_pred, loss_type, batch_size, n_speaker, n_output, pit_axis=1):
    # [batch, spk #, length]
    real_spk_num = n_speaker

    # TODO 1: # output channel != # speaker
    v_perms = tf.constant(list(itertools.permutations(range(n_output), n_speaker)))
    v_perms_onehot = tf.one_hot(v_perms, n_output)

    y_true_exp = tf.expand_dims(y_true, pit_axis+1) # [batch, n_speaker, 1,        len]
    y_pred_exp = tf.expand_dims(y_pred, pit_axis)   # [batch, 1,         n_output, len]

    cross_total_loss = get_loss(loss_type, y_true_exp, y_pred_exp)

    loss_sets = tf.einsum('bij,pij->bp', cross_total_loss, v_perms_onehot) 
    loss = tf.reduce_min(loss_sets, axis=1)
    loss = tf.reduce_mean(loss)
        
    # find permutation sets for y pred
    s_perm_sets = tf.argmin(loss_sets, 1)
    s_perm_choose = tf.gather(v_perms, s_perm_sets)
    s_perm_idxs = tf.stack([
        tf.tile(
            tf.expand_dims(tf.range(batch_size), 1),
            [1, n_speaker]),
        s_perm_choose], axis=2)

    s_perm_idxs = tf.reshape(s_perm_idxs, [batch_size*n_speaker, 2])
    y_pred = tf.gather_nd(y_pred, s_perm_idxs)
    y_pred = tf.reshape(y_pred, [batch_size, n_speaker, -1])

    if loss_type != 'sdr':
        sdr = evaluation.sdr(y_true[:,:real_spk_num,:], y_pred[:,:real_spk_num,:])
        sdr = tf.reduce_mean(sdr)
    else:
        sdr = -loss/n_speaker

    return loss, y_pred, sdr, s_perm_choose

def get_loss(loss_type, t_true_exp, t_pred_exp, axis=-1):
    if loss_type == 'l1':
        y_cross_loss = t_true_exp - t_pred_exp
        cross_total_loss = tf.reduce_sum(tf.abs(y_cross_loss), axis=axis)

    elif loss_type == 'l2':
        y_cross_loss = t_true_exp - t_pred_exp
        cross_total_loss = tf.reduce_sum(tf.square(y_cross_loss), axis=axis)

    elif loss_type == 'snr':
        cross_total_loss = -evaluation.snr(t_true_exp, t_pred_exp)

    elif loss_type == 'sdr':
        cross_total_loss = -evaluation.sdr(t_true_exp, t_pred_exp)

    elif loss_type == 'sisnr':
        cross_total_loss = -evaluation.sisnr(t_true_exp, t_pred_exp)

    elif loss_type == 'sdr_modify':
        cross_total_loss = -evaluation.sdr_modify(t_true_exp, t_pred_exp)

    elif loss_type == 'sisdr':
        cross_total_loss = -evaluation.sisdr(t_true_exp, t_pred_exp)

    elif loss_type == 'sym_sisdr':
        cross_total_loss = -evaluation.sym_sisdr(t_true_exp, t_pred_exp)

    return cross_total_loss

In [7]:
loss, y_pred, sdr, s_perm_choose = pit_loss(s, s_est, 'sisnr', BATCH_SIZE, 2, 2, pit_axis=1)
print(loss)
print(y_pred)
print(sdr)
print(s_perm_choose)

tf.Tensor(-37.075935, shape=(), dtype=float32)
tf.Tensor(
[[[3.2 4.5 4.1]
  [1.4 1.8 1.7]]

 [[2.8 2.  4.7]
  [1.4 2.3 4.2]]], shape=(2, 2, 3), dtype=float32)
tf.Tensor(21.44663, shape=(), dtype=float32)
tf.Tensor(
[[1 0]
 [0 1]], shape=(2, 2), dtype=int32)


In [8]:
def sisnri(s, s_est, mix_s, batch_size, n_speaker, n_output, pit_axis=1):
    mix_s = tf.repeat(mix_s,n_speaker,axis=1) 
    mix_s = tf.reshape(mix_s,(batch_size,n_speaker,-1))
    
    loss, _, _, _ = pit_loss(s, s_est,  'sisnr', batch_size, n_speaker, n_output, pit_axis=1)
    loss_b, _, _, _ = pit_loss(s, mix_s, 'sisnr', batch_size, n_speaker, n_output, pit_axis=1)
    if tf.math.is_nan(loss_b):
        return loss*-1

    loss *= -1
    loss_b *= -1
    return loss-loss_b

In [9]:
loss = sisnri(s, s_est, mix_s,  BATCH_SIZE, 2, 2, pit_axis=1)
print(loss)

tf.Tensor(37.075935, shape=(), dtype=float32)


In [10]:
def sdri(s, s_est, mix_s, batch_size, n_speaker, n_output, pit_axis=1):
    mix_s = tf.repeat(mix_s,n_speaker,axis=1) 
    mix_s = tf.reshape(mix_s,(batch_size,n_speaker,-1))
    
    loss, _, sdr, _ = pit_loss(s, s_est,  'sdr', batch_size, n_speaker, n_output, pit_axis=1)
    loss_b, _, sdr_b, _ = pit_loss(s, mix_s, 'sdr', batch_size, n_speaker, n_output, pit_axis=1)
    
    if tf.math.is_nan(sdr_b):
        return sdr   
    
    return sdr-sdr_b

In [11]:
loss = sdri(s, s_est, mix_s, BATCH_SIZE, 2, 2, pit_axis=1)
print(loss)

tf.Tensor(8.873157, shape=(), dtype=float32)
